In [1]:
# import os
# import subprocess
# import tempfile
# from langchain_community.agent_toolkits.github.toolkit import GitHubToolkit
# from langchain_community.utilities.github import GitHubAPIWrapper
# from langchain_openai import ChatOpenAI

# class GitHubRepoSetup:
#     def __init__(self, repo_url, target_directory=None):
#         """
#         Initialize GitHub repository setup
        
#         Args:
#             repo_url (str): URL of the GitHub repository
#             target_directory (str, optional): Directory to clone repository. 
#                                               Defaults to a temporary directory.
#         """
#         self.repo_url = repo_url
#         self.target_directory = target_directory or tempfile.mkdtemp()
        
#         # Initialize GitHub toolkit (optional, for additional GitHub interactions)
#         github = GitHubAPIWrapper()
#         self.toolkit = GitHubToolkit.from_github_api_wrapper(github)
        
#         # Initialize language model
#         self.llm = ChatOpenAI(temperature=0, model="gpt-4o")

#     def clone_repository(self):
#         """
#         Clone the GitHub repository to the target directory
#         """
#         try:
#             subprocess.run([
#                 'git', 'clone', self.repo_url, self.target_directory
#             ], check=True)
#             print(f"Repository cloned to: {self.target_directory}")
#             return self.target_directory
#         except subprocess.CalledProcessError as e:
#             print(f"Error cloning repository: {e}")
#             return None

#     def detect_project_type(self):
#         """
#         Detect the type of project based on existing files
#         """
#         project_indicators = {
#             'Python': ['requirements.txt', 'pyproject.toml', 'setup.py'],
#             'Node.js': ['package.json'],
#             'Java': ['pom.xml', 'build.gradle'],
#             'Ruby': ['Gemfile'],
#             'Rust': ['Cargo.toml']
#         }

#         for lang, files in project_indicators.items():
#             for file in files:
#                 if os.path.exists(os.path.join(self.target_directory, file)):
#                     return lang
        
#         return 'Unknown'

#     def setup_virtual_environment(self, project_type):
#         """
#         Set up a virtual environment based on project type
#         """
#         try:
#             if project_type == 'Python':
#                 # Create Python virtual environment
#                 subprocess.run([
#                     'python3', '-m', 'venv', 
#                     os.path.join(self.target_directory, 'venv')
#                 ], check=True)
                
#                 # Activate virtual environment and install dependencies
#                 venv_python = os.path.join(self.target_directory, 'venv', 'bin', 'python')
#                 venv_pip = os.path.join(self.target_directory, 'venv', 'bin', 'pip')
                
#                 # Check for different dependency files
#                 if os.path.exists(os.path.join(self.target_directory, 'requirements.txt')):
#                     subprocess.run([
#                         venv_pip, 'install', 
#                         '-r', os.path.join(self.target_directory, 'requirements.txt')
#                     ], check=True)
#                 elif os.path.exists(os.path.join(self.target_directory, 'pyproject.toml')):
#                     subprocess.run([
#                         venv_pip, 'install', 'poetry'
#                     ], check=True)
#                     subprocess.run([
#                         'poetry', 'install'
#                     ], cwd=self.target_directory, check=True)
            
#             elif project_type == 'Node.js':
#                 # Use nvm to manage Node.js version
#                 subprocess.run([
#                     'npm', 'install'
#                 ], cwd=self.target_directory, check=True)
            
#             print(f"Virtual environment and dependencies set up for {project_type} project")


# Simple code executor

In [2]:
! pip install langchain_experimental 

Defaulting to user installation because normal site-packages is not writeable


In [ ]:
# from langchain_openai import ChatOpenAI
# from langchain_experimental.utilities import PythonREPL
# from langchain_core.messages import (
#     BaseMessage, 
#     HumanMessage,
#     AIMessage
# )
# from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
# from langchain_core.tools import tool
# from tenacity import retry, stop_after_attempt
# import json

# # Initialize Python REPL
# repl = PythonREPL()

# @tool
# @retry(stop=stop_after_attempt(3))
# def docker_ubuntu_terminal(
#     code: str,
# ):
#     """Use this to execute python code. If you want to see the output of a value,
#     you should print it out with `print(...)`. This is visible to the user.
#     Always give the code in the required JSON format."""
#     print("repl initiated")
#     try:
#         result = repl.run(code)
#     except BaseException as e:
#         return f"Failed to execute. Error: {repr(e)}"
#     result_str = f"\nSuccessfully executed:\n```python\n{code}\n```\nStdout: {result}"
#     return (
#         result_str + "\n\nIf you have completed all tasks, respond with FINAL ANSWER."
#     )

# class CodeExecutionAgent:
#     def __init__(self,api_key="oh-no"):
#         self.llm = ChatOpenAI(
#             model="gpt-4o",
#             temperature=0,
#             api_key=api_key
#         )
        
#         # Create the prompt template
#         self.prompt = ChatPromptTemplate.from_messages([
#             ("system", "You are a helpful AI assistant that can execute Python code. "
#                       "Use the docker_ubuntu_terminal tool to execute code and help solve problems. "
#                       "Always show your thinking process and explain what the code does."),
#             MessagesPlaceholder(variable_name="messages")
#         ])
        
#         # Create the chain
#         self.agent = self.prompt | self.llm.bind_tools([docker_ubuntu_terminal])
    
#     def run(self, query: str):
#         """Run a query through the agent"""
#         messages = [HumanMessage(content=query)]
#         response = self.agent.invoke({"messages": messages})
#         return response.content

# # Usage example
# if __name__ == "__main__":
#     agent = CodeExecutionAgent(api_key="api-key")
    
#     while True:
#         query = input("Enter your question (or 'exit' to quit): ")
#         if query.lower() == 'exit':
#             break
            
#         try:
#             response = agent.run(query)
#             print("\nResponse:", response)
#         except Exception as e:
#             print(f"An error occurred: {str(e)}")

'2106.14834'

In [4]:
import requests

def download_arxiv_pdf(arxiv_url, output_path):
    response = requests.get(arxiv_url)
    if response.status_code == 200:
        with open(output_path, 'wb') as f:
            f.write(response.content)
        print(f"PDF downloaded successfully and saved to {output_path}")
    else:
        print(f"Failed to download PDF. Status code: {response.status_code}")

arxiv_url = "https://arxiv.org/pdf/2308.03688.pdf"
output_path = f"{arxiv_url[-14:-4]}.pdf"
download_arxiv_pdf(arxiv_url, output_path)

PDF downloaded successfully and saved to 2308.03688.pdf


In [ ]:
%pip install google-genai

Defaulting to user installation because normal site-packages is not writeable


In [10]:
from PyPDF2 import PdfReader

text = ""

reader = PdfReader(output_path)
number_of_pages = len(reader.pages)

for n in range(number_of_pages):
    page = reader.pages[n]
    text += page.extract_text()

print(text)

Technical Report (v0.2)
AGENT BENCH : EVALUATING LLM S AS AGENTS
Xiao Liu1,*, Hao Yu1,*, Hanchen Zhang1, Yifan Xu1, Xuanyu Lei1, Hanyu Lai1, Yu Gu2,
Hangliang Ding1, Kaiwen Men1, Kejuan Yang1, Shudan Zhang1, Xiang Deng2, Aohan Zeng1,
Zhengxiao Du1, Chenhui Zhang1, Sheng Shen3, Tianjun Zhang3, Yu Su2, Huan Sun2,
Minlie Huang1, Yuxiao Dong1, Jie Tang1
1Tsinghua University,2The Ohio State University,3UC Berkeley
ABSTRACT
Large Language Models (LLMs) are becoming increasingly smart and autonomous,
targeting real-world pragmatic missions beyond traditional NLP tasks. As a result,
there has been an urgent need to evaluate LLMs as agents on challenging tasks
in interactive environments. We present AGENT BENCH , a multi-dimensional
evolving benchmark that currently consists of 8 distinct environments to assess
LLM-as-Agent’s reasoning and decision-making abilities in a multi-turn open-
ended generation setting. Our extensive test over 27 API-based and open-sourced
(OSS) LLMs shows that, while 

In [ ]:
from google import genai

client = genai.Client(api_key="google-api")
response = client.models.generate_content(
    model="gemini-2.0-flash", contents=f"I will give you a text of a research paper, if they have provided a link to a github repo of their work in the paper, give it to me, if there is no such repo link provided in the paper, reply with 'no repo found', paper: {text}"
)
print(response.text)


```
https://github.com/THUDM/AgentBench
```


In [1]:
from autogen.coding import DockerCommandLineCodeExecutor, CodeBlock
import tempfile

# Create a temporary directory to store the code files.
temp_dir = tempfile.TemporaryDirectory()

# Create a Docker command line code executor.
executor = DockerCommandLineCodeExecutor(
    image="ubuntu",  # Execute code using the given docker image name.
    timeout=420,  # Timeout for each code execution in seconds.
    work_dir=temp_dir.name,  # Use the temporary directory to store the code files.
)

blk0 = CodeBlock(
    code="apt-get update",
    language="bash"
)

blk1 = CodeBlock(
    code="apt-get install -y python3 pip",
    language="bash"
)

blk2 = CodeBlock(
    code="python3 --version",
    language="bash"
)

blk3 = CodeBlock(
    code="pip install json",
    language="bash"
)

#execute the code
result = executor.execute_code_blocks(code_blocks=[blk0])
print(result)
result = executor.execute_code_blocks(code_blocks=[blk1])
print(result)
result = executor.execute_code_blocks(code_blocks=[blk2])
print(result)
result = executor.execute_code_blocks(code_blocks=[blk3])
print(result)

# def python_docker(code:str):
#     try:
#         blk = CodeBlock(
#             code=code,
#             language="bash"
#         )
#         result = executor.execute_code_blocks(code_blocks=[blk])
#         result_str = f"\nSuccessfully executed:\n```python\n{code}\n```\nStdout: {result}"
#         return (
#         result_str + "\n\nIf you have completed all tasks, respond with FINAL ANSWER."
#         )
#     except Exception as e:
#         return f"Failed to execute. Error: {repr(e)}"

Patching name='__init__', member=<function LLMLingua.__init__ at 0x7df5559abd80>, patched=<function function.__call__ at 0x7df5559abce0>
Patching name='compress_text', member=<function LLMLingua.compress_text at 0x7df5559abe20>, patched=<function function.__call__ at 0x7df5559e0040>
exit_code=0 output='Get:1 http://security.ubuntu.com/ubuntu noble-security InRelease [126 kB]\nGet:2 http://archive.ubuntu.com/ubuntu noble InRelease [256 kB]\nGet:3 http://security.ubuntu.com/ubuntu noble-security/universe amd64 Packages [1057 kB]\nGet:4 http://archive.ubuntu.com/ubuntu noble-updates InRelease [126 kB]\nGet:5 http://archive.ubuntu.com/ubuntu noble-backports InRelease [126 kB]\nGet:6 http://security.ubuntu.com/ubuntu noble-security/restricted amd64 Packages [916 kB]\nGet:7 http://archive.ubuntu.com/ubuntu noble/main amd64 Packages [1808 kB]\nGet:8 http://security.ubuntu.com/ubuntu noble-security/multiverse amd64 Packages [34.0 kB]\nGet:9 http://security.ubuntu.com/ubuntu noble-security/main

In [ ]:
# while True:
#     code = input()
#     if code == "exit":
#         break
#     blk = CodeBlock(
#         code=code,
#         language="bash"
#     )
#     result = executor.execute_code_blocks(code_blocks=[blk])
#     print(result)

In [ ]:
# First we initialize the model we want to use.
from langchain_openai import ChatOpenAI
from langchain_core.messages import (
    BaseMessage, 
    HumanMessage,
    AIMessage
)
from langchain_core.tools import tool
from tenacity import retry, stop_after_attempt


model = ChatOpenAI(model="gpt-4o", temperature=0, api_key="my-asshair")

from langchain_core.tools import tool


@tool
@retry(stop=stop_after_attempt(10))
def docker_ubuntu_terminal(
    code: str
):
    """This is an ubuntu command line in a docker container, use this to execute command line instructions. use apt-get install -y to install packages. Write and execute python code by writing to and executing a file. Use pip install to install python packages, for example, to install the json package, the command is: 'pip install json' never use exclamation marks when issuing commands like git or pip or any other command, use pip instead of !pip and git instead of !git."""
    try:
        blk = CodeBlock(
            code=code,
            language="sh"
        )
        result = executor.execute_code_blocks(code_blocks=[blk])
        result_str = f"\nSuccessfully executed code:\n```sh\n{code}\n```\nStdout: {result}"
        return (
        result_str + "\n\nIf you have completed all tasks, respond with FINAL ANSWER."
        )
    except Exception as e:
        return f"Failed to execute.: {repr(e)}"

tools = [docker_ubuntu_terminal]

# Define the graph

from langgraph.prebuilt import create_react_agent

graph = create_react_agent(model, tools=tools)


In [4]:
def print_stream(stream):
    for s in stream:
        message = s["messages"][-1]
        if isinstance(message, tuple):
            print(message)
        else:
            message.pretty_print()

In [ ]:
inputs = {"messages": [("user", "clone the repo https://github.com/lucidrains/rectified-flow-pytorch, and follow the instructions in the readme file")]}

print_stream(graph.stream(inputs, stream_mode="values"))

================================ Human Message =================================

clone the repo https://github.com/lucidrains/rectified-flow-pytorch, and follow the instructions in the readme file
================================== Ai Message ==================================
Tool Calls:
  docker_ubuntu_terminal (call_4mixnabdXxAZqI4z9mpiL35B)
 Call ID: call_4mixnabdXxAZqI4z9mpiL35B
  Args:
    code: git clone https://github.com/lucidrains/rectified-flow-pytorch.git
================================= Tool Message =================================
Name: docker_ubuntu_terminal


Successfully executed code:
```sh
git clone https://github.com/lucidrains/rectified-flow-pytorch.git
```
Stdout: exit_code=128 output="fatal: destination path 'rectified-flow-pytorch' already exists and is not an empty directory.\n" code_file='/tmp/tmp3f6bqmxd/tmp_code_40ea4d568fa16ec54cb0206bcf9f9d75.sh'

If you have completed all tasks, respond with FINAL ANSWER.
================================== Ai Message =